![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Análisis de sentimientos y técnicas de NLP

En este taller podrán poner en práctica sus conocimientos sobre las diferentes técnicas para el procesamiento de lenguaje natural. El taller está constituido por 5 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción sentimientos de viajeros en Twitter

En este taller se usará el conjunto de datos de sentimientos sobre distintas aerolíneas de EE.UU. provenientes de Twitter. Cada observación contiene si el sentimiento de los tweets es positivo, neutral o negativo teniendo en cuenta distintas variables como aerolínea y las razones de los sentimientos negativos (como "retraso en el vuelo" o "servicio grosero"). El objetivo es predecir el sentimiento asociado a cada tweet. Para más detalles pueden visitar el siguiente enlace: [datos](https://www.kaggle.com/crowdflower/twitter-airline-sentiment).

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación de librerías
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# Lectura de la información de archivo .zip
tweets = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/Tweets.zip', index_col=0)

# Visualización dataset
tweets.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
tweet_id,,,,,,,,,,,,,,
570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
# Impresión tamaño del cojunto de datos
tweets.shape

(14640, 14)

### Análisis descriptivo

In [5]:
# Cuenta de tweets por cada sentimiento
tweets['airline_sentiment'].value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [6]:
# Cuenta de tweets por cada aerolínea
tweets['airline'].value_counts()

airline
United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: count, dtype: int64

In [7]:
# Plot con cuenta de tweets por cada aerolínea y sentimiento
pd.crosstab(index = tweets["airline"],columns = tweets["airline_sentiment"]).plot(kind='bar',figsize=(10, 6),alpha=0.5,rot=0,stacked=True,title="Sentiminetos por aerolínea")

<Axes: title={'center': 'Sentiminetos por aerolínea'}, xlabel='airline'>

### Liberias y Variables de interés y predicción

In [26]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
import nltk



In [9]:
# Separación de variables predictoras (X) y de variable de interés (y)
X = tweets['text']
y = tweets['airline_sentiment'].map({'negative':-1,'neutral':0,'positive':1})

In [10]:
# Separación de datos en set de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Uso de CountVectorizer

En la celda 1 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función **CountVectorizer** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [11]:
# Entrenamiento y predicción con Random Forest
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

rf_count = RandomForestClassifier(n_estimators=100, random_state=42)
rf_count.fit(X_train_count, y_train)
y_pred_count = rf_count.predict(X_test_count)

In [12]:
class_report_rf = classification_report(y_test, y_pred_count, output_dict=True)
confusion_matrix_rf = confusion_matrix(y_test, y_pred_count)
accuracy_rf = accuracy_score(y_test, y_pred_count)
print(f"Accuracy Random Forest con CountVectorizer:")
print(class_report_rf)
print(f"Confussion Matrix Random Forest con CountVectorizer:")
print(confusion_matrix_rf)
print(f"Accuracy Random Forest con CountVectorizer: {accuracy_rf:.4f}")

Accuracy Random Forest con CountVectorizer:
{'-1': {'precision': 0.7757765596450013, 'recall': 0.9633711507293355, 'f1-score': 0.8594563331405437, 'support': 3085.0}, '0': {'precision': 0.6915077989601387, 'recall': 0.4054878048780488, 'f1-score': 0.5112107623318386, 'support': 984.0}, '1': {'precision': 0.8372641509433962, 'recall': 0.46526867627785057, 'f1-score': 0.5981465880370682, 'support': 763.0}, 'accuracy': 0.7711092715231788, 'macro avg': {'precision': 0.7681828365161788, 'recall': 0.6113758772950783, 'f1-score': 0.6562712278364835, 'support': 4832.0}, 'weighted avg': {'precision': 0.7683251051017005, 'recall': 0.7711092715231788, 'f1-score': 0.747276495145983, 'support': 4832.0}}
Confussion Matrix Random Forest con CountVectorizer:
[[2972   90   23]
 [ 539  399   46]
 [ 320   88  355]]
Accuracy Random Forest con CountVectorizer: 0.7711


### Punto 2 - Eliminación de Stopwords

En la celda 2 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función CountVectorizer, **eliminen stopwords** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [13]:
# Celda 2


### Punto 3 - Lematización con verbos

En la celda 3 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función CountVectorizer, **lematizen el texto con verbos** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [14]:
# Celda 3


### Punto 4 - Multiples técnicas

En la celda 4 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función **CountVectorizer, eliminen stopwords, lematizen el texto con verbos** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [32]:
# Creamos una función que limpie el texto
# Quitamos palabras vacías y lematizamos las palabras restantes
lemmatizer = WordNetLemmatizer()  # Inicializamos el lematizador
def limpiar_texto(texto):
    stop_words = set(stopwords.words('english'))  # Lista de stopwords 
    tokens = wordpunct_tokenize(texto)  # Separamos el texto por palabras (tokens)
    
    # Filtramos las palabras: que no sean stopwords y que sean letras (no signos)
    tokens_filtrados = [p for p in tokens if p.lower() not in stop_words and p.isalpha()]
    
    # Lematizamos usando como referencia los verbos
    lematizadas = [lemmatizer.lemmatize(p, pos='v') for p in tokens_filtrados]
    
    # Unimos todo nuevamente en un texto limpio
    return ' '.join(lematizadas)

# Aplicamos esta limpieza a los datos de entrenamiento y prueba
X_train_limpio = X_train.apply(limpiar_texto)
X_test_limpio = X_test.apply(limpiar_texto)

# Convertimos los textos a vectores usando CountVectorizer
vectorizador = CountVectorizer()
X_train_vec = vectorizador.fit_transform(X_train_limpio)
X_test_vec = vectorizador.transform(X_test_limpio)

In [31]:
# Creamos y entrenamos el modelo Random Forest
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train_vec, y_train)

# Hacemos predicciones y evaluamos el modelo
predicciones = modelo_rf.predict(X_test_vec)

# Métricas de desempeño
accuracy = accuracy_score(y_test, predicciones)
reporte = classification_report(y_test, predicciones)
matriz_conf = confusion_matrix(y_test, predicciones)

# Mostramos los resultados
print(" Accuracy del modelo combinado:", accuracy)
print("\n Reporte de clasificación:\n", reporte)
print("\n Matriz de confusión:\n", matriz_conf)


 Accuracy del modelo combinado: 0.7731788079470199

 Reporte de clasificación:
               precision    recall  f1-score   support

          -1       0.81      0.92      0.86      3085
           0       0.62      0.45      0.52       984
           1       0.74      0.61      0.67       763

    accuracy                           0.77      4832
   macro avg       0.72      0.66      0.68      4832
weighted avg       0.76      0.77      0.76      4832


 Matriz de confusión:
 [[2829  181   75]
 [ 453  445   86]
 [ 204   97  462]]


### Punto 5 - Comparación y análisis de resultados

En la celda 5 comparen los resultados obtenidos de los diferentes modelos (random forest) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [ ]:
# Celda 5
